In [1]:
import numpy as np
import pandas as pd

In [8]:
stats = pd.read_csv('../data/lol_pro_stats/pro_stats.csv')

In [9]:
stats.columns

Index(['Team', 'Player', 'Season', 'Champion', 'Game', 'Win', 'Lose',
       'WinRate', 'Kill', 'Death', 'Assist', 'KDA', 'VS', 'DMG', 'DPM', 'DTPM',
       'CS', 'CSPM', 'KPAR', 'DPAR', 'KS', 'Gold', 'GPM', 'Base'],
      dtype='object')

In [10]:
# 필요한 열만 추출

stats = stats[['Player', 'Season', 'Champion', 'Game', 'WinRate', 'KDA', 'DMG', 'DPM', 'DTPM', 'CSPM', 'KPAR', 'DPAR', 'KS', 'GPM']]
stats.head()

,Player,Season,Champion,Game,WinRate,KDA,DMG,DPM,DTPM,CSPM,KPAR,DPAR,KS,GPM
0,Kiin,2019 LCK Spring,Aatrox,7,42.90%,2.60,11.4k,330,646,9.26,36.80%,24.40%,20.20%,393
1,Kiin,2019 LCK Spring,Akali,4,50.00%,3.80,15.7k,365,604,7.55,61.10%,18.60%,34.00%,393
2,Kiin,2019 LCK Spring,Ryze,4,25.00%,1.23,20.4k,537,467,8.47,56.00%,29.70%,15.30%,373
3,Kiin,2019 LCK Spring,Lucian,3,33.30%,1.77,23.5k,642,583,9.24,59.80%,36.80%,31.30%,414
4,Kiin,2019 LCK Spring,Vladimir,3,66.70%,6.25,16.9k,502,692,9.32,49.50%,30.50%,22.20%,434
